In [9]:
from espn_api.basketball import League, Matchup, Team

In [10]:
espn_s2 = "AECc%2BKMx5L8l9Rai5PHG%2FT%2Fjtw%2BR%2FZozX391uxpta1J6tD61%2F4NV88%2BVPlCSe0Wjcg0J9jJ0oTJ%2BMkehJLT8MSDfgiubWB9F1n3H%2BnTHWc9OTkzM2ceQFGPiA%2B9AF4yQzt0I3%2FdCgk%2BdQLbgybhS6y1yRS9NIDyfXMSgcqFqRbn5F7zBTzwu9A%2BD2Pi6g6KMR29PbmNbtxSrCJkfbF%2BaZtrpqDbHBZhLyfqOAhQIDx0q1iD0u1DkhxVNoRnxx%2FmEvS%2FHEJZWmfvZM%2F0IAHxPrfVx"
swid="{3C4A75B6-F84B-48AE-8A75-B6F84B48AE47}"

In [11]:
league = League(league_id=30695, year=2021, espn_s2=espn_s2, swid=swid)

In [12]:
MY_TEAM_ID = 2
schedule = league.get_team_data(MY_TEAM_ID).schedule

In [13]:
league.recent_activity()

[Activity((Team(Shanghai SHARKS),DROPPED,Spencer Dinwiddie) (Team(Shanghai SHARKS),FA ADDED,Isaac Okoro)),
 Activity((Team(Shanghai SHARKS),FA ADDED,Larry Nance Jr.)),
 Activity((Team(Shanghai SHARKS),TRADED,Deandre Ayton) (Team(SEA MoNeYPro),TRADED,Rui Hachimura) (Team(SEA MoNeYPro),TRADED,Khris Middleton) (Team(Shanghai SHARKS),TRADED,Dejounte Murray)),
 Activity((Team(PHI Lower Merion 33),FA ADDED,Davis Bertans)),
 Activity((Team(PHI Lower Merion 33),DROPPED,Aron Baynes)),
 Activity((Team(PHI Lower Merion 33),FA ADDED,Aron Baynes)),
 Activity((Team(PHI Lower Merion 33),DROPPED,Joe Harris)),
 Activity((Team(SEA MoNeYPro),DROPPED,Marc Gasol) (Team(SEA MoNeYPro),FA ADDED,Nicolas Batum)),
 Activity((Team(Team Beyond),FA ADDED,Will Barton) (Team(Team Beyond),DROPPED,Taurean Prince)),
 Activity((Team(CZ ButtKicker),FA ADDED,Patrick Beverley) (Team(CZ ButtKicker),DROPPED,De'Andre Hunter)),
 Activity((Team(Team Beyond),FA ADDED,Taurean Prince)),
 Activity((Team(SEA MoNeYPro),FA ADDED,Marc G

In [14]:
from activity.myUtil import get_opponent_team, get_my_team

In [15]:
opponent_roster = get_opponent_team(schedule[0], MY_TEAM_ID).roster

In [16]:
opponent_roster

[Player(Giannis Antetokounmpo),
 Player(Devin Booker),
 Player(Jimmy Butler),
 Player(Jamal Murray),
 Player(CJ McCollum),
 Player(Steven Adams),
 Player(Michael Porter Jr.),
 Player(Dennis Schroder),
 Player(Mike Conley),
 Player(Tyrese Haliburton),
 Player(Mason Plumlee),
 Player(Derrick Rose),
 Player(RJ Barrett),
 Player(DeAndre Jordan)]

In [17]:
from activity.score_predicter import predict, cumulate_number_of_games

In [18]:
number_of_games_map = cumulate_number_of_games(1, league)

In [19]:
predict(opponent_roster, number_of_games_map)

(1100.0701226430133, 1242.6163335317165)

In [20]:
my_roster = get_my_team(schedule[0], MY_TEAM_ID).roster
predict(my_roster, number_of_games_map)

(1180.500690167327, 1334.899662604407)

In [21]:
predicted_points_team_name_map = {}
for team in league.teams:
    predicted_points_team_name_map[team.team_name] = predict(team.roster, number_of_games_map)

In [22]:
def get_tuple_average(tup):
    return (tup[0] + tup[1]) / 2

In [23]:
table_output = []
for team in league.teams:
    predicted_points = predicted_points_team_name_map[team.team_name]
    table_output.append((team.team_name, predicted_points[0], predicted_points[1], get_tuple_average(predicted_points)))
table_output.sort(reverse=True, key=lambda x: x[-1])

In [24]:
from IPython.display import HTML, display
import tabulate
table_output.insert(0, ("Team Name", "Low", "High", "Avg"))
display(HTML(tabulate.tabulate(table_output, tablefmt='html')))

Team Name,Low,High,Avg
PHI Lower Merion 33,1410.1773854158382,1482.748298712264,1446.462842064051
Team Beyond,1313.3915973541614,1466.3691659101803,1389.8803816321708
Badwomen ``,1295.3500333404713,1385.1622088867505,1340.2561211136108
NY Zhanos Diesel,1248.6800008482837,1383.8665503695336,1316.2732756089085
Team Tao,1247.6748290498388,1372.5776807053428,1310.1262548775908
Dragon City Tiedan,1239.459076532539,1345.0435120714355,1292.2512943019874
SEA MoNeYPro,1180.500690167327,1334.899662604407,1257.700176385867
Shanghai SHARKS,1205.6664446108325,1301.7358766852499,1253.7011606480412
Tim Liang,1100.0701226430133,1242.6163335317165,1171.3432280873649
CZ ButtKicker,1104.743789053202,1221.5616610541367,1163.1527250536692


In [25]:
match_up_points = []
match_up_points.append(["Home Team", "Estimate Points", "Away Team", "Estimate Points", "+/-"])
for matchup in league.scoreboard():
    home_team_average = get_tuple_average(predict(matchup.home_team.roster, number_of_games_map))
    away_team_average = get_tuple_average(predict(matchup.away_team.roster, number_of_games_map))
    match_up_points.append([matchup.home_team.team_name, home_team_average,
                            matchup.away_team.team_name, away_team_average,
                           home_team_average - away_team_average])

In [26]:
display(HTML(tabulate.tabulate(match_up_points, tablefmt='html')))

Home Team,Estimate Points,Away Team,Estimate Points,+/-
PHI Lower Merion 33,1446.462842064051,CZ ButtKicker,1163.1527250536692,283.3101170103819
Tim Liang,1171.3432280873649,SEA MoNeYPro,1257.700176385867,-86.35694829850218
NY Zhanos Diesel,1316.2732756089085,Dragon City Tiedan,1292.2512943019874,24.021981306921134
Team Beyond,1389.8803816321708,Team Tao,1310.1262548775908,79.75412675458006
Badwomen ``,1340.2561211136108,Shanghai SHARKS,1253.7011606480412,86.5549604655696
